In [14]:
import skimage
from skimage import io,morphology,measure
import os
import numpy as np

srcs = ['/home/kuki/Desktop/novo/active_set/Train/ANN','/home/kuki/Desktop/novo/active_set/Valid/ANN']
for src in srcs[0:1]:
    labels = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('.tif')]
    for label in labels:
        mask = skimage.io.imread(label)
        # mask = cv2.resize(mask,dsize=(256,256))
        # convert 2D greyscale mask where unique pixel values represent different tissue types
        # into 3D boolean mask where each channel is a tissue type
        mask_layers = np.empty((len(np.unique(mask)),)+mask.shape)
        class_ids = []
        connected_region_layers = np.empty(mask.shape+(1,))
        classes = [_ for _ in np.unique(mask) if _ != 0]
        for class_id in classes:
            mask_layer_closed = skimage.morphology.area_closing(mask==class_id)
            mask_layer_opened = skimage.morphology.area_opening(mask_layer_closed)
            labeled_mask_layer = skimage.measure.label(mask_layer_opened, background=0)
            # iterate different connected regions
            regions = [_ for _ in np.unique(labeled_mask_layer) if _ != 0]
            for region in regions:
                connected_region_layer = labeled_mask_layer==region
                connected_region_layers = np.dstack(
                    (connected_region_layers,connected_region_layer))
                class_ids.append(class_id)
        connected_region_layers = connected_region_layers[:,:,1:]
        class_ids = np.array(class_ids)
        dst = label.replace('ANN','ANN_stack')
        dst1 = dst.replace('.tif','_mask.npy')
        dst2 = dst.replace('.tif','_ids.npy')
        np.save(dst1,connected_region_layers.astype(np.bool))
        np.save(dst2,class_ids.astype(np.int32))

In [15]:
import numpy as np
asd=np.load('/home/kuki/Desktop/novo/active_set/Train/ANN_stack/3067mask.npy')

FileNotFoundError: [Errno 2] No such file or directory: '/home/kuki/Desktop/novo/active_set/Train/ANN_stack/3067mask.npy'